In [2]:
import psutil

notebook_memory = psutil.virtual_memory()

if notebook_memory.total < 32 * 1024 * 1024:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True
    print(notebook_memory)

svmem(total=33242578944, available=30943203328, percent=6.9, used=1884225536, free=18929790976, active=1334169600, inactive=11108872192, buffers=2768896, cached=12425793536, shared=942080, slab=1380466688)


In [4]:
%pip install ray==2.3.0


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import Dataset
import ray

lm_dataset_train = Dataset.from_parquet('./data-gpt3/gpt3-train/*.parquet')
#lm_dataset_validation = Dataset.from_parquet('./data-gpt3/gpt3-validation/*.parquet')
#lm_dataset_test = Dataset.from_parquet('./data-gpt3/gpt3-test/*.parquet')

ray_train_ds = ray.data.from_huggingface(
    lm_dataset_train
)

# ray_validation_ds = ray.data.from_huggingface(
#     lm_dataset_validation
# )

# ray_test_ds = ray.data.from_huggingface(
#     lm_dataset_test
# )

Using custom data configuration default-f369d81bb38cb6ef


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-f369d81bb38cb6ef/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


2023-03-15 04:23:09,887	WARNING services.py:1791 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 4044349440 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=9.67gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-03-15 04:23:10,013	INFO worker.py:1553 -- Started a local Ray instance.


AttributeError: module 'pickle' has no attribute 'PickleBuffer'

In [3]:
import ray
ray.shutdown()

from ray.train.huggingface import HuggingFaceTrainer
from ray.air.config import ScalingConfig, RunConfig
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from ray.tune import SyncConfig


def trainer_init_per_worker(train_dataset, 
                            eval_dataset, 
                            **config):

    model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
    
    args = TrainingArguments(
        output_dir=f"{model_checkpoint}-finetuned-amazon-customer-reviews",
#        evaluation_strategy="epoch",
#        save_strategy="epoch",
#        logging_strategy="epoch",
        learning_rate=2e-5,
        weight_decay=0.01,  
        max_steps=10,
#        eval_steps=10,
        num_train_epochs=1.0,
        no_cuda=True        
    )
    
    return Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
#        eval_dataset=eval_dataset,
    )


s3_checkpoint_prefix='./tmp_checkpoint' #"s3://dsoaws/ray_output"

import multiprocessing
num_cpus = multiprocessing.cpu_count()

scaling_config = ScalingConfig(num_workers=1, 
                               #use_gpu=True,
                               trainer_resources={"CPU": num_cpus - 2}, #, "GPU": 0},
                               #_max_cpu_fraction_per_node = 0.8
                              )

# If using GPUs, use the below scaling config instead.
# scaling_config = ScalingConfig(num_workers=3, use_gpu=True)
trainer = HuggingFaceTrainer(
    trainer_init_per_worker=trainer_init_per_worker,
    scaling_config=scaling_config,
    run_config = RunConfig(
        sync_config=SyncConfig(
            # This will store checkpoints in S3.
            upload_dir=s3_checkpoint_prefix
        )
    ),
    datasets={"train": ray_train_ds, 
              #"evaluation": ray_evaluation_ds
             },
)
result = trainer.fit()

2023-02-26 20:28:17,566	WARNING services.py:1791 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 3999264768 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=9.78gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-02-26 20:28:17,605	INFO worker.py:1550 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


ValueError: An application is trying to access a Ray object whose owner is unknown(00ffffffffffffffffffffffffffffffffffffff0100000001000000). Please make sure that all Ray objects you are trying to access are part of the current Ray session. Note that object IDs generated randomly (ObjectID.from_random()) or out-of-band (ObjectID.from_binary(...)) cannot be passed as a task argument because Ray does not know which task created them. If this was not how your object ID was generated, please file an issue at https://github.com/ray-project/ray/issues/

In [ ]:
result.checkpoint

In [ ]:
TODO.save_pretrained('s3://dsoaws/bloom/output/model_from_notebook_ray/')

In [ ]:
import transformers
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bigscience/bloom-560m', use_fast=True)
#model = AutoModelForCausalLM.from_pretrained('s3://dsoaws/bloom/output/model_from_notebook_ray/')
                                             

In [ ]:
type(trainer.trainer_init_per_worker.model)

In [ ]:
prompt = "Write a review for Norton Antivirus"
result_length = 100
inputs = tokenizer(prompt, return_tensors="pt")

print(inputs)


In [ ]:
print(tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=result_length, 
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))